In [2]:
import pandas as pd
import pandas_datareader.data as web
import datetime
from pathlib import Path
import numpy as np
#import pandas_datareader as pdr
import hvplot.pandas
import seaborn as sns
import holoviews as hv
import matplotlib.pyplot as plt


import scipy.optimize as sco
plt.style.use('fivethirtyeight')
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

AttributeError: module 'holoviews' has no attribute 'core'

In [ ]:
#Read data files

quandl_file = Path("Resources/Quandl_data.csv")
asset_file = Path("Resources/asset_prices.csv")
bitcoin_file = Path("Resources/bitcoin_data.csv")

alt_data_df = pd.read_csv(quandl_file, index_col = "yr_mo", infer_datetime_format = True, parse_dates = True)
asset_prices_df = pd.read_csv(asset_file, index_col = "Date", infer_datetime_format = True, parse_dates = True)
bitcoin_df = pd.read_csv(bitcoin_file, index_col = "Date" , infer_datetime_format = True, parse_dates = True)

bitcoin_df.drop(columns = ["Open*", "High", "Low", "Volume", "Market Cap"], inplace=True)
bitcoin_df.columns = ["Bitcoin"]
bitcoin_df.sort_index()

asset_prices_df.drop(columns = "BLOK", inplace= True)
asset_prices_df = pd.merge(asset_prices_df, bitcoin_df, how = "inner", on= "Date" )
asset_prices_df.sort_index(inplace=True)
asset_prices_df["Bitcoin"]= asset_prices_df["Bitcoin"].str.replace(",", "").astype(float)

In [ ]:
alt_data_df.head()

In [ ]:
asset_prices_df.dropna(inplace=True)
asset_prices_df.head()

In [ ]:
asset_returns_monthly_df = asset_prices_df[asset_prices_df.index.day == 1].pct_change()
asset_returns_monthly_df.dropna(inplace=True)
asset_returns_monthly_df.head()

In [ ]:
asset_returns_df = asset_prices_df.pct_change().dropna()
asset_returns_df.head()

In [ ]:
# standard deviation
assets_st_dev_19 = asset_returns_df[(asset_returns_df.index >= '2019-01-01') & (asset_returns_df.index < '2020-01-01') ].std()
assets_st_dev_19 = pd.DataFrame(assets_st_dev_19, columns = ['STD_19'] )

assets_st_dev_20 = asset_returns_df[asset_returns_df.index >= '2020-01-01'].std()
assets_st_dev_20 = pd.DataFrame(assets_st_dev_20, columns = ['STD_20'] )

#assets_st_dev_19

In [ ]:
# annualized standard deviation
annualized_st_dev = asset_returns_df.std() * np.sqrt(252)
annualized_st_dev = pd.DataFrame(annualized_st_dev, columns = ['Annualized_STD'] )
#annualized_st_dev

In [ ]:
#sharpe ratios

sharpe_ratios_df = (asset_returns_df.mean() * 252) / (asset_returns_df.std() * np.sqrt(252))
sharpe_ratios_df = pd.DataFrame( sharpe_ratios_df, columns = ['Sharpe_Ratios'] )
#sharpe_ratios_df


In [ ]:
#Average return over the whole period
mean_daily_return_df_19 = pd.DataFrame(asset_returns_df[(asset_returns_df.index>='2019-01-01') & 
                                                        (asset_returns_df.index<'2020-01-01')].mean(),
                                                         columns = ['Avg_daily_return_2019'])

mean_daily_return_df_20 = pd.DataFrame(asset_returns_df[(asset_returns_df.index>='2020-01-01')].mean(),
                                                         columns = ['Avg_daily_return_2020'])

In [ ]:
#BETA FUNCTION

def calculate_beta(data, market_variable = "SPY"):
    
    beta_df = pd.DataFrame(index=[1])
    
    for i in data.columns:
        
        variance = data[i].var()
        covariance = data[i].cov(data[market_variable])
        beta = covariance / variance

        beta_df[i]= beta
    
    return beta_df.T



In [ ]:
#beta represents the volatility compared to the market, beta more then 1 means the asset is more riskier

beta_df_19 = calculate_beta(asset_returns_df[(asset_returns_df.index>='2019-01-01') & 
                                            (asset_returns_df.index<'2020-01-01')], "SPY")
beta_df_19.columns = ['Beta_19']

beta_df_20 = calculate_beta(asset_returns_df[(asset_returns_df.index>='2020-01-01')], "SPY")
beta_df_20.columns = ['Beta_30']
#beta_df

In [ ]:
asset_returns_monthly_df[asset_returns_monthly_df.index>'2019-01-01'].hvplot(title = "Asset Returns 2019-2020")

In [ ]:
all_data = pd.merge(asset_returns_monthly_df, alt_data_df, how="inner",left_index = True, right_index=True)
all_data.dropna(inplace = True)
all_data.drop(columns = ["SP500_pe_shiller", "SP500_eyield", "15y_mort_int", 'SP500_pe','Unemployment_LT','GDP_R'], inplace=True)
all_data

In [ ]:
correlation = all_data.corr()
sns.heatmap(correlation,vmin=-1, vmax=1, linewidths = .2)

In [ ]:
def get_rsi_timeseries(prices, n=14):
    # RSI = 100 - (100 / (1 + RS))
    # where RS = (Wilder-smoothed n-period average of gains / Wilder-smoothed n-period average of -losses)
    # Note that losses above should be positive values
    # Wilder-smoothing = ((previous smoothed avg * (n-1)) + current value to average) / n
    # For the very first "previous smoothed avg" (aka the seed value), we start with a straight average.
    # Therefore, our first RSI value will be for the n+2nd period:
    #     0: first delta is nan
    #     1:
    #     ...
    #     n: lookback period for first Wilder smoothing seed value
    #     n+1: first RSI

    # First, calculate the gain or loss from one price to the next. The first value is nan so replace with 0.
    deltas = (prices-prices.shift(1)).fillna(0)

    # Calculate the straight average seed values.
    # The first delta is always zero, so we will use a slice of the first n deltas starting at 1,
    # and filter only deltas > 0 to get gains and deltas < 0 to get losses
    avg_of_gains = deltas[1:n+1][deltas > 0].sum() / n
    avg_of_losses = -deltas[1:n+1][deltas < 0].sum() / n

    # Set up pd.Series container for RSI values
    rsi_series = pd.Series(0.0, deltas.index)

    # Now calculate RSI using the Wilder smoothing method, starting with n+1 delta.
    up = lambda x: x if x > 0 else 0
    down = lambda x: -x if x < 0 else 0
    i = n+1
    
    for d in deltas[n+1:]:
        avg_of_gains = ((avg_of_gains * (n-1)) + up(d)) / n
        avg_of_losses = ((avg_of_losses * (n-1)) + down(d)) / n
        
        if avg_of_losses != 0:
            rs = avg_of_gains / avg_of_losses
            rsi_series[i] = 100 - (100 / (1 + rs))
        else:
            rsi_series[i] = 100
        i += 1

    return rsi_series

In [ ]:
rsi_df = pd.DataFrame()

for i in asset_prices_df.columns :
    rsi_df[i] = get_rsi_timeseries(asset_prices_df[i])
    

rsi_df[rsi_df.index >'2020-03-01'].hvplot()                                 



In [ ]:
#KPI table
KPI_df = pd.merge(mean_daily_return_df_19, mean_daily_return_df_20, how ="inner", left_index = True, right_index=True)
KPI_df = pd.merge(KPI_df, assets_st_dev_19,how ="inner", left_index = True, right_index=True)
KPI_df = pd.merge(KPI_df, assets_st_dev_20,how ="inner", left_index = True, right_index=True)
#KPI_df = pd.merge(KPI_df , annualized_st_dev, how ="inner", left_index = True, right_index=True)
KPI_df = pd.merge(KPI_df, sharpe_ratios_df ,  how ="inner", left_index = True, right_index=True)
KPI_df = pd.merge(KPI_df, beta_df_19 ,  how ="inner", left_index = True, right_index=True)
KPI_df = pd.merge(KPI_df, beta_df_20 ,  how ="inner", left_index = True, right_index=True)
KPI_df["RSI"] = rsi_df.iloc[-1]
#volume


KPI_df

In [ ]:
#Logarythmic returns
log_ret = np.log(asset_prices_df[asset_prices_df.index > '2019-01-01']/asset_prices_df[asset_prices_df.index > '2019-01-01'].shift(1))
log_ret.head()

In [ ]:
log_ret.hvplot.hist(bins=100, subplots=True, width=500, group_label='Ticker', grid=True).cols(3)

In [ ]:
#efficient portfolio frontier plot

In [ ]:
num_ports = 10000

all_weights = np.zeros((num_ports,len(asset_returns_df.columns)))
ret_arr = np.zeros(num_ports)
vol_arr = np.zeros(num_ports)
sharpe_arr = np.zeros(num_ports)

for ind in range(num_ports):

    # Create Random Weights
    weights = np.array(np.random.random(12))

    # Rebalance Weights
    weights = weights / np.sum(weights)
    
    # Save Weights
    all_weights[ind,:] = weights

    # Expected Return
    ret_arr[ind] = np.sum((log_ret.mean() * weights) *252)

    # Expected Variance
    vol_arr[ind] = np.sqrt(np.dot(weights.T, np.dot(log_ret.cov() * 252, weights)))

    # Sharpe Ratio
    sharpe_arr[ind] = ret_arr[ind]/vol_arr[ind]

In [ ]:
#look for th epoint with the highest sharpe ratio

max_sr_ret = ret_arr[sharpe_arr.argmax()]
max_sr_vol = vol_arr[sharpe_arr.argmax()]

In [ ]:
scatter = hv.Scatter((vol_arr, ret_arr, sharpe_arr), 'Volatility', ['Return', 'Sharpe Ratio'])
max_sharpe = hv.Scatter([(max_sr_vol,max_sr_ret)])

scatter.opts(color='Sharpe Ratio', cmap='plasma', width=600, height=400, colorbar=True, padding=0.1) *\
max_sharpe.opts(color='red', line_color='black', size=10)

In [ ]:
def get_ret_vol_sr(weights):
    """
    Takes in weights, returns array or return,volatility, sharpe ratio
    """
    weights = np.array(weights)
    ret = np.sum(log_ret.mean() * weights) * 252
    vol = np.sqrt(np.dot(weights.T, np.dot(log_ret.cov() * 252, weights)))
    sr = ret/vol
    return np.array([ret,vol,sr])


def neg_sharpe(weights):
    return  get_ret_vol_sr(weights)[2] * -1

In [ ]:
from scipy.optimize import minimize
#help.minimize

# Contraints
def check_sum(weights):
    '''
    Returns 0 if sum of weights is 1.0
    '''
    return np.sum(weights) - 1

# By convention of minimize function it should be a function that returns zero for conditions
cons = ({'type':'eq','fun': check_sum})

# 0-1 bounds for each weight
bounds = ((0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1))

# Initial Guess (equal distribution)
init_guess = [1,0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

# Sequential Least SQuares Programming (SLSQP).
opt_results = minimize(neg_sharpe,init_guess,method='SLSQP',bounds=bounds,constraints=cons)

In [ ]:
# Create a linspace number of points to calculate x on
frontier_y = np.linspace(0,0.25,20) # Change 100 to a lower number for slower computers!]

def minimize_volatility(weights):
    return  get_ret_vol_sr(weights)[1] 
frontier_volatility = []

for possible_return in frontier_y:
    # function for return
    
    cons = ({'type':'eq','fun': check_sum},
            {'type':'eq','fun': lambda w: get_ret_vol_sr(w)[0] - possible_return})
    
    result = minimize(minimize_volatility,init_guess,method='SLSQP',bounds=bounds,constraints=cons)
    
    frontier_volatility.append(result['fun'])
    
scatter * hv.Curve((frontier_volatility, frontier_y)).opts(color='green', line_dash='dashed')

In [ ]:
get_ret_vol_sr(opt_results.x)

In [ ]:
sharpe_arr.argmax()

In [ ]:
#Optional portfolio


KPI_df['Optimal_Portfolio'] = all_weights[sharpe_arr.argmax(),:]

KPI_df

In [ ]:
#optimal portfolio test


In [ ]:

def portfolio_annualised_performance(weights, mean_returns, cov_matrix):
    returns = np.sum(mean_returns*weights ) *252
    std = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights))) * np.sqrt(252)
    return std, returns
  
def random_portfolios(num_portfolios, mean_returns, cov_matrix, risk_free_rate):
    results = np.zeros((3,num_portfolios))
    weights_record = []
    for i in range(num_portfolios):
        weights = np.random.random(12)
        weights /= np.sum(weights)
        weights_record.append(weights)
        portfolio_std_dev, portfolio_return = portfolio_annualised_performance(weights, mean_returns, cov_matrix)
        results[0,i] = portfolio_std_dev
        results[1,i] = portfolio_return
        results[2,i] = (portfolio_return - risk_free_rate) / portfolio_std_dev
    return results, weights_record

def display_simulated_ef_with_random(mean_returns, cov_matrix, num_portfolios, risk_free_rate):
    results, weights = random_portfolios(num_portfolios,mean_returns, cov_matrix, risk_free_rate)
    
    max_sharpe_idx = np.argmax(results[2])
    sdp, rp = results[0,max_sharpe_idx], results[1,max_sharpe_idx]
    max_sharpe_allocation = pd.DataFrame(weights[max_sharpe_idx],index=asset_returns_df.columns,columns=['allocation'])
    max_sharpe_allocation.allocation = [round(i*100,2)for i in max_sharpe_allocation.allocation]
    max_sharpe_allocation = max_sharpe_allocation.T
    
    min_vol_idx = np.argmin(results[0])
    sdp_min, rp_min = results[0,min_vol_idx], results[1,min_vol_idx]
    min_vol_allocation = pd.DataFrame(weights[min_vol_idx],index=asset_returns_df.columns,columns=['allocation'])
    min_vol_allocation.allocation = [round(i*100,2)for i in min_vol_allocation.allocation]
    min_vol_allocation = min_vol_allocation.T
    
    print ("-"*80)
    print ("Maximum Sharpe Ratio Portfolio Allocation\n")
    print ("Annualised Return:", round(rp,2))
    print ("Annualised Volatility:", round(sdp,2))
    print ("\n")
    print (max_sharpe_allocation)
    print ("-"*80)
    print ("Minimum Volatility Portfolio Allocation\n")
    print ("Annualised Return:", round(rp_min,2))
    print ("Annualised Volatility:", round(sdp_min,2))
    print ("\n")
    print (min_vol_allocation)
    
    plt.figure(figsize=(10, 7))
    plt.scatter(results[0,:],results[1,:],c=results[2,:],cmap='YlGnBu', marker='o', s=10, alpha=0.3)
    plt.colorbar()
    plt.scatter(sdp,rp,marker='*',color='r',s=500, label='Maximum Sharpe ratio')
    plt.scatter(sdp_min,rp_min,marker='*',color='g',s=500, label='Minimum volatility')
    plt.title('Simulated Portfolio Optimization based on Efficient Frontier')
    plt.xlabel('annualised volatility')
    plt.ylabel('annualised returns')
    plt.legend(labelspacing=0.8)

In [ ]:
returns = asset_returns_df
mean_returns = returns.mean()
cov_matrix = returns.cov()
num_portfolios = 25000
risk_free_rate = asset_returns_df['STIP'].iloc[-1]


display_simulated_ef_with_random(mean_returns, cov_matrix, num_portfolios, risk_free_rate)